# Task 1 — EDA

This notebook performs initial exploration of the provided unified dataset and creates a small set of candidate enrichment records. All plots use the project's `Plotter` utility (`src.utils.plotter`).

In [32]:
# Imports and setup
import sys
from pathlib import Path
import pandas as pd

# Add src to path for imports
sys.path.insert(0, "../")

from src.utils.plotter import Plotter

# Initialize plotter
plotter = Plotter(figures_dir=Path("../outputs/figures"))